<a href="https://colab.research.google.com/github/asrafulasf72/Data-Mining-Algorithm/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
def gini_impurity(labels):
    """Calculate Gini impurity for a list of class labels."""
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / counts.sum()
    return 1 - np.sum(probabilities ** 2)

In [ ]:
def information_gain(left_labels, right_labels, current_impurity):
    """Calculate information gain after splitting."""
    total = len(left_labels) + len(right_labels)
    p_left = len(left_labels) / total
    p_right = len(right_labels) / total

    return current_impurity - (p_left * gini_impurity(left_labels) +
                               p_right * gini_impurity(right_labels))

In [ ]:
def split_dataset(X, y, feature_index, threshold):
    """Split dataset based on a feature threshold."""
    left_indices = np.where(X[:, feature_index] <= threshold)[0]
    right_indices = np.where(X[:, feature_index] > threshold)[0]
    return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

In [ ]:
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))

        # Stop conditions (leaf node)
        if (depth == self.max_depth or
            num_classes == 1 or
            num_samples < self.min_samples_split):
            leaf_value = self._majority_class(y)
            return TreeNode(value=leaf_value)

        best_feature, best_threshold, best_gain = None, None, -1
        current_impurity = gini_impurity(y)

        # Search best split
        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                gain = information_gain(y_left, y_right, current_impurity)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        # No useful split found
        if best_gain == -1:
            leaf_value = self._majority_class(y)
            return TreeNode(value=leaf_value)

        # Make recursive splits
        X_left, y_left, X_right, y_right = split_dataset(X, y, best_feature, best_threshold)
        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return TreeNode(best_feature, best_threshold, left_child, right_child)

    def _majority_class(self, y):
        values, counts = np.unique(y, return_counts=True)
        return values[np.argmax(counts)]

    # Prediction for one sample
    def _predict_one(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    # Prediction for all samples
    def predict(self, X):
        return np.array([self._predict_one(sample, self.root) for sample in X])


# ----------------------------------------------
# Example usage
# ----------------------------------------------
if __name__ == "__main__":
    # Simple dataset
    X = np.array([
        [2.7, 2.5],
        [1.3, 3.3],
        [3.1, 1.8],
        [1.0, 1.0],
        [3.0, 3.0]
    ])
    y = np.array([0, 0, 1, 1, 0])

    clf = DecisionTreeClassifier(max_depth=3)
    clf.fit(X, y)

    predictions = clf.predict(X)
    print("Predictions:", predictions)

Predictions: [0 0 1 1 0]
